<img src="https://electronaut.info/wp-content/uploads/2017/05/Tubescape_jupyter2.jpg">

<h1><font color = b30000>TUBE DATA MATCHER for uTRACER FILES</font></h1><h5>Anode current for a sweep of grid voltages  when anode voltage = constant</h5>


# STEP 1 — Import and process tube data, build dataframe.

In [1]:
from electronaut_uTracer_Import import *
df.head()

Please enter the path to the folder containing the uTracer files: /Users/rrmc/github_rrMacKinnon/Electrona_uTracer/SampleTubeData_1_to_196

What type of tubes are these? 5749

— — — — — 

Data from 167 tubes of type 5749 was successfully read.
A dataframe containing all 167 tubes was successfully created.


,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,1,5749,325,99.278,65.383,44.717,34.082,26.669,20.931,16.492,12.992,10.291,8.23,6.669,5.495,4.626
1,10,5749,325,84.662,55.752,37.5,27.666,20.873,15.84,12.088,9.304,7.261,5.807,4.747,3.945,3.344
2,100,5749,325,76.506,50.319,33.695,25.027,19.052,14.603,11.283,8.775,6.954,5.655,4.765,4.118,3.619
3,102,5749,325,92.106,59.906,39.752,29.517,22.626,17.543,13.745,10.894,8.787,7.228,6.08,5.204,4.575
4,103,5749,325,69.639,45.449,30.772,22.843,17.624,13.782,10.864,8.615,6.916,5.657,4.754,4.092,3.619


##### TODO:  INSERT PLOT OF ALL TUBES


# STEP 3: Show the statistics of the tube data set

In [2]:
# For some reason, .describe is requiring the .astype(float) in order to show the stats.
df_stats = df.astype(float).describe()
df_stats.loc[:, 'tube_type'] = df_stats.loc[:, 'tube_type'].astype(int)
df_stats.loc[:, 'tube_ID'] = df_stats.loc[:, 'tube_ID'].astype(int)
# df_stats.loc[:, 'tube_ID'] = df_stats.loc[:, 'tube_ID'].astype(str)
df_stats

,tube_ID,tube_type,anode_voltage,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
count,167,167,167.0,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000,167.000000
mean,94,5749,325.0,85.854024,54.600138,35.936389,26.606778,20.321790,15.648988,12.173521,9.577371,7.657347,6.248994,5.208695,4.410192,3.798886
std,57,0,0.0,10.252616,5.708263,3.792364,2.876220,2.387349,2.053135,1.794487,1.576286,1.382130,1.206177,1.052825,0.929768,0.840940
min,1,5749,325.0,48.494000,30.962000,19.730000,14.837000,11.473000,8.866000,6.827000,5.260000,4.091000,3.235000,2.613000,2.153000,1.833000
25%,43,5749,325.0,80.143500,51.841500,34.041500,25.167500,19.052000,14.428000,11.082500,8.645000,6.855500,5.501500,4.584500,3.869500,3.319500
50%,94,5749,325.0,87.154000,55.362000,36.408000,26.777000,20.534000,15.759000,12.180000,9.602000,7.679000,6.162000,5.096000,4.343000,3.712000
75%,143,5749,325.0,91.902000,58.625500,38.192000,28.252000,21.554500,16.662500,13.070000,10.400000,8.427000,6.923000,5.801500,4.957500,4.302500
max,196,5749,325.0,107.892000,65.654000,46.996000,36.666000,30.080000,25.292000,21.460000,18.384000,15.898000,13.867000,12.198000,10.760000,9.616000


##### TODO:  INSERT PLOT(S) OF STATISTICS 

# STEP 4: Calculate the differences between all tubes

In [3]:
def build_difference_df(df, df_stats):
       
    # Make a list of all the columns in the dataframe
    cols = list(df.columns)
    
    # Make a list of just the bias columns
    bias_list = []
    for i in cols:
        if i.startswith('Bias_') == True:
            bias_list.append(i)
    
    # BMake an empty list to hold the calculated values, to be used to build the difference dataframe
    diff_squared_list = []
    
    # Iterate through the index of tubes in the master dataframe to select a reference tube
    for each_ref_tube in df.index:
        
        # Get the next reference tube data, as a series
        ref_tube = df.iloc[each_ref_tube]
        ref_tube_ID = ref_tube.tube_ID

        # Iterate through the index of tubes in the master dataframe to select a match tube
        for each_match_tube in df.index:
            
            # Get the next tube data to be compared, as a series
            match_tube = df.iloc[each_match_tube]
            match_tube_ID = match_tube.tube_ID
            
            # Dictionary container to hold the difference variables for each tube
            temp_dict = {}
            
            # Compute the squares of the differences in each current measurement
            error_sum = 0
            for bias in enumerate(bias_list):
                mismatch = ((float(match_tube.loc[bias[1]]) - float(ref_tube.loc[bias[1]]))**2)
                temp_dict[bias[1]] = mismatch
                error_sum = error_sum + mismatch

            # Add other relevant key/value pairs to the dictionary  
            temp_dict['ref_tube_ID'] = ref_tube_ID
            temp_dict['match_tube_ID'] = match_tube_ID
            temp_dict['error_sum'] = error_sum

            # Append the tube's calculated values to the diff_squared_list
            diff_squared_list.append(temp_dict)
    
    # Make an ordered list for the columns of the new difference dataframe
    new_cols_to_add = ['ref_tube_ID', 'match_tube_ID', 'error_sum']
    df_dif_cols = new_cols_to_add + bias_list
    df_dif = pd.DataFrame(diff_squared_list, columns = df_dif_cols)
    return df_dif
    
    
df_dif = build_difference_df(df, df_stats)
df_dif.head()

,ref_tube_ID,match_tube_ID,error_sum,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
0,1,1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,10,514.934156,213.627456,92.756161,52.085089,41.165056,33.593616,25.918281,19.395216,13.601344,9.180900,5.870929,3.694084,2.402500,1.643524
2,1,100,1116.246220,518.563984,226.924096,121.484484,81.993025,58.018689,40.043584,27.133681,17.783089,11.135569,6.630625,3.625216,1.896129,1.014049
3,1,102,170.397792,51.437584,29.997529,24.651225,20.839225,16.345849,11.478544,7.546009,4.401604,2.262016,1.004004,0.346921,0.084681,0.002601
4,1,103,1805.026199,878.470321,397.364356,194.463025,126.315121,81.812025,51.108201,31.674384,19.158129,11.390625,6.620329,3.667225,1.968409,1.014049


# Find the best matches for a particular tube_ID

In [12]:
def find_best_set_for_chosen_tube_ID(tube_to_match, tube_set_size):
    tube_to_match = str(tube_to_match)
    
    # Create a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
    df_tube_set = df_dif[(df_dif.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(tube_set_size)
    
    return df_tube_set

tube_to_match = 17
tube_set_size = 16
find_best_set_for_chosen_tube_ID(tube_to_match, tube_set_size)

,ref_tube_ID,match_tube_ID,error_sum,Bias_2,Bias_6,Bias_10,Bias_14,Bias_18,Bias_22,Bias_26,Bias_30,Bias_34,Bias_38,Bias_42,Bias_46,Bias_50
10920,17,17,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
10958,17,32,0.104842,0.010404,0.007225,0.001681,0.016384,0.000049,0.013924,0.021316,0.016641,0.008464,0.002601,0.002500,0.002209,0.001444
10976,17,5,0.739507,0.436921,0.165649,0.011881,0.047089,0.040000,0.020449,0.011664,0.003364,0.000484,0.000144,0.000121,0.000841,0.000900
10877,17,123,1.034648,0.048400,0.839056,0.097344,0.018225,0.007744,0.000400,0.005041,0.009025,0.007396,0.001936,0.000049,0.000016,0.000016
10892,17,14,1.860642,0.506944,0.719104,0.119716,0.006561,0.012544,0.025281,0.007225,0.000361,0.022801,0.063504,0.105625,0.129600,0.141376
10903,17,15,2.718726,1.449616,0.048400,0.069696,0.188356,0.298116,0.304704,0.198916,0.099225,0.035344,0.003364,0.001936,0.009604,0.011449
10882,17,13,2.744267,0.372100,1.002001,0.316969,0.028900,0.003721,0.006724,0.064516,0.146689,0.221841,0.243049,0.193600,0.109561,0.034596
10941,17,194,2.762397,2.226064,0.152100,0.139129,0.007744,0.032041,0.042849,0.047524,0.036864,0.027556,0.023104,0.008836,0.000361,0.018225
10871,17,117,2.790934,1.575025,0.748225,0.405769,0.018496,0.000441,0.000016,0.000100,0.000289,0.001764,0.004900,0.006724,0.012544,0.016641
11016,17,94,3.282925,1.140624,1.104601,0.567009,0.171396,0.068121,0.042436,0.016641,0.002916,0.002116,0.020736,0.040401,0.049284,0.056644


# This is ALMOST working lol:

In [32]:
# Find the best set of tubes for a given tube_ID and return a dictionary of the tubes and their score
# The returned dictionary value will then be used to add a row in a dataframe containing the tube sets and scores

def find_best_set_for_chosen_tube_ID(tube_to_match, tube_set_size, tube_sets_columns):
    tube_to_match = str(tube_to_match)
        
    # Create a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
    df_tube_set = df_dif[(df_dif.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(tube_set_size)
    
    # Create a series of the tube set tube_IDs to be used later when dropping them from 
    series_of_tubes_in_set = df_tube_set.match_tube_ID
    tube_set_list = list(series_of_tubes_in_set)
    
     
    # Create the dictionary of the tubes in the set and their total score
    tube_set_dict = dict(zip(tube_sets_columns, tube_set_list))
    
    tube_set_score = df_tube_set.error_sum.sum()
    tube_set_dict.update({'tube_set_score':tube_set_score})

    return tube_set_dict, tube_sets_columns

tube_list = list(df_dif.ref_tube_ID.unique().astype(str))
tube_sets_and_scores_list = []


def ask_tube_set_size():
    
    # Ask the user to enter the set size
    tube_set_size = input("Please enter the number of tubes per matched set:")
    
    # Build a list of tube names for the tube set size
    tube_sets_columns = ['Tube_' + str(1 + i) for i in range(int(tube_set_size))]    
    
    return tube_set_size, tube_sets_columns 

tube_set_size_info = ask_tube_set_size()
tube_set_size = tube_set_size_info[0]
tube_sets_columns = tube_set_size_info[1]

# for i in tube_list:
temp = find_best_set_for_chosen_tube_ID('22', tube_set_size, tube_sets_columns)
#     tube_set_dict = temp[0]
#     tube_sets_and_scores_list.append(temp_list)

df_tube_sets = pd.DataFrame(tube_sets_and_scores_list, columns=tube_sets_columns)
df_tube_sets



Please enter the number of tubes per matched set:6


TypeError: cannot do slice indexing on <class 'pandas.indexes.numeric.Int64Index'> with these indexers [6] of <class 'str'>

In [ ]:
temp_tube_set_series = df_tube_set.match_tube_ID
tube_set_total_error = df_tube_set.error_sum.sum()
print('Tube Set Score =',tube_set_total_error, ' (lower is better)')

In [ ]:
tube_set_size = 8
tube_list = list(df_dif.ref_tube_ID.unique().astype(str))



tube_to_match = str(tube_list[0])
print("The first chosen tube is:", tube_to_match)
df_temp_tube_set = df_dif[(df_dif.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(tube_set_size)


# Creating a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
df_tube_set


# STEP 5: Find the best matches for any tube set size

In [ ]:
# Define a function that takes a tube set size, then calculates the best tube sets from the data set

# After computing the first best tube set, those tubes are removed from the dataset
# The calculation repeats for each tube until the dataset is depleted

# Passing the entire difference dataframe into a function seems like a bad idea.  What's a better way to do this?

def determine_best_matches(df_dif):
    
    def find_matches_to(df_dif):
    
    # Make a list of all the tubes
    tube_list = list(df_dif.ref_tube_ID.unique().astype(str))
    
    # Ask the desired tube set size 
    tube_set_size = input("How many tubes per set? ")
    
    # Build a list of columns for each tube in the set size
    tube_sets_columns = ['Tube_' + str(1 + i) for i in range(int(tube_set_size))]
    
    tube_set_score_list = []
#     for i in tube_list:
    # Filter the matches for each tube in tube_list
    df_tube_set = df_dif[(df_dif.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(tube_set_size)

    temp_tube_set = list(df_tube_set.match_tube_ID)
    temp_tube_set_score = df_tube_set.error_sum.sum()

    return df_tube_set
    
# Enter the tube to be matched and the size of the matched set
# tube_to_match = 2

# tube_set_size = 8

# tube_to_match = str(tube_to_match)

# Creating a new dataframe showing only tube_set_size number of nearest matches to the chosen tube_to_match
# df_tube_set = df_dif[(df_dif.ref_tube_ID == tube_to_match)].sort_values('error_sum').head(tube_set_size)
# df_tube_set

determine_best_matches(df_dif)

# df_tube_set = df_tube_set.sort_values('error_sum').head(tube_set_size)
df_tube_set

In [ ]:
tube_set_score = df_tube_set.error_sum.sum()
print("Tube S8et Score:", tube_set_score)

In [ ]:
import ipywidgets as wg
from ipywidgets import Layout, Box
from IPython.display import display


# Make a list of all the tube_ID names, then build a select box with those values
tube_list = list(df.loc[:, 'tube_ID'])
tube_list.sort(key=int)
tube_ID_select_box = wg.Select(options=tube_list, value=tube_list[0], description='Reference Tube', layout=Layout(width='150px'), disabled=False)

# Make a list of the tubes for the "Matched Tubes" selectMultiple box
matched_list = ['1', '33', '67', '22', '45']
matched_tube_box = wg.SelectMultiple(options=matched_list, layout=Layout(display="flex", flex_flow='column', width='350px'), description="Matched Tubes") 


items = [tube_ID_select_box, matched_tube_box]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box = Box(children=items, layout=box_layout)

box


# def respond(sender):
    # pass the chosen Tube_ID to a function that looks up the resulting match list, then fill the match list box
    
    
# display(tube_ID_select_box,matched_tube_box)
# tube_ID_select_box.observe(respond)

### Make a function that receives tube_to_match and tube_set_size arguments, adds up the total of all the error_sum values per set for a set score, then sorts and displays all set scores.

### Consider adding a column called "available", and any matched sets can be removed from the remaining dataset by toggling the availability to False.

# Miscellaneous stuff

##### Apply a function to a column
df.Bias_2.max()

In [ ]:
# Applymap is useful for applying a function to all data
# For example, converting all data from floats to ints
# df.loc[2:7, 'Bias_2': 'Bias_50'].applymap(int)

In [ ]:
# List all available magics
% lsmagic

In [ ]:
# Display environmental variables
env_variables = %env
current_path = env_variables.get('PATH')
current_path

In [ ]:
mydict = % env
mydict.keys()

In [ ]:
% env

In [ ]:
# % time x = range(10000)
% timeit x = range(10000)
max(x)